In [1]:
!pip install --user nltk

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
cd ./drive/MyDrive/git_hub_repo/NLP_tasks/NLP/Homework_2/ngram_authorship_train/


/content/drive/MyDrive/git_hub_repo/NLP_tasks/NLP/Homework_2/ngram_authorship_train


In [34]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends, flatten
from nltk.lm import MLE
from nltk.util import bigrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.lm import Laplace

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [205]:
f = open('austen_utf8.txt', 'r')
text_austen = f.read()

f = open('dickens_utf8.txt', 'r')
text_dickens = f.read()

f = open('tolstoy_utf8.txt', 'r')
text_tolstoy = f.read()

f =open('wilde_utf8.txt', 'r')
text_wilde = f.read()

In [181]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [163]:
def preprocess_text(text):
  sents = sent_tokenize(text, language='english')

  clean_sents = []
  for sent in sents:
    clean_sents.append(clean_text(sent))

  clean_words = []
  for sent in clean_sents:
    clean_words.append(word_tokenize(sent))

  return clean_words

In [164]:
def split_train_test(clean_words):
  train_size = len(clean_words) * 0.8
  train_sentences = clean_words[:int(train_size)]
  test_sentences = clean_words[int(train_size):]
  return train_sentences, test_sentences

In [165]:
def train_lm(train_words, n):
  train, vocab = padded_everygram_pipeline(n, train_words)
  lm = Laplace(n)
  lm.fit(train, vocab)
  return lm

In [186]:
def test_lm(lm, test_words):
  test_paddedline  = list(bigrams(list(pad_both_ends(test_words, n=2))))
  return lm.perplexity(test_paddedline)

In [227]:
austen_clean_words = preprocess_text(text_austen)
dickens_clean_words = preprocess_text(text_dickens)
tolstoy_clean_words = preprocess_text(text_tolstoy)
wilde_clean_words = preprocess_text(text_wilde)


austen_train, austen_test = split_train_test(austen_clean_words)
austen_lm = train_lm(austen_train, 3)

dickens_train, dickens_test = split_train_test(dickens_clean_words)
dickens_lm = train_lm(dickens_train, 3)

tolstoy_train, tolstoy_test = split_train_test(tolstoy_clean_words)
tolstoy_lm = train_lm(tolstoy_train, 3)

wilde_train, wilde_test = split_train_test(wilde_clean_words)
wilde_lm = train_lm(wilde_train, 3)

In [221]:
def show_accuracy_lm(author):

  if author == 'austen':
    test_set = austen_test

  elif author == 'dickens':
    test_set = dickens_test

  elif author == 'tolstoy':
    test_set = tolstoy_test

  elif author == 'wilde':
    test_set = wilde_test



  true_or_false_list = []

  for test_sent in test_set:

    austen_perplexity = test_lm(austen_lm, test_sent)
    dickens_perplexcity = test_lm(dickens_lm, test_sent)
    tolstoy_perplexity = test_lm(tolstoy_lm, test_sent)
    wilde_perplexity = test_lm(wilde_lm, test_sent)

    compares = {'austen': austen_perplexity,
                'dickens': dickens_perplexcity,
                'tolstoy': tolstoy_perplexity,
                'wilde': wilde_perplexity}

    min_perplexity = min(compares, key=compares.get)


    if author == min_perplexity:

      true_or_false_list.append(1)

    else:
      true_or_false_list.append(0)



  return sum(true_or_false_list)/len(true_or_false_list)

In [228]:
print('austen_accuracy:', show_accuracy_lm('austen'))
print('dickens_accuracy', show_accuracy_lm('dickens'))
print('tolstoy_accuracy', show_accuracy_lm('tolstoy'))
print('wilde_accuracy', show_accuracy_lm('wilde'))




austen_accuracy: 0.6995841995841996
dickens_accuracy 0.6022044088176353
tolstoy_accuracy 0.34433113377324537
wilde_accuracy 0.7014519056261344
